# TEMPLATE JAWABAN TUGAS LANGUAGE MODELING

### Import Library

In [29]:
import nltk
import xlrd
import operator
import random
import math
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Import Dataset (Training dan Testing)

In [30]:
book = xlrd.open_workbook('dataTrainArticle.xlsx')
book2 = xlrd.open_workbook('dataTestArticle.xlsx')
sheet = book.sheet_by_name('Sheet1')
sheet2 = book2.sheet_by_name('Sheet1')
dataTrainDefault = [[sheet.cell_value(r, c) for c in range(sheet.ncols)] for r in range(sheet.nrows)]
dataTestDefault = [[sheet2.cell_value(r, c) for c in range(sheet2.ncols)] for r in range(sheet2.nrows)]
#split each sentence in each article
dataTrain = [] #init
for article in dataTrainDefault:
    sentence = article[0].split('.')
    for word in sentence:
        dataTrain.append(word)
dataTest = [] #init
counter = 0
for article in dataTestDefault:
    sentence = article[0].split('.')
    for word in sentence:
        dataTest.append(word)
#add dummy-ish data for OOV (word that doesn't exist in data train), the OOV will be converted to _____
#just 4 basic word classes, noun, verbs, adjectives, adverb
#without this p(oov) will be 0
dataTrain.append('_____ has _____ the _____') 
#<noun> has <verb> the <noun>
dataTrain.append('_____  is one _____ car')
#<noun> is one <adjective> car
dataTrain.append('_____  is _____ _____ a new car')
#<noun> is <adverb> <noun> a new car
print('done')

done


### Bangun Model Unigram

In [31]:
def buildUnigramModel(trainingSentences):
    #init var
    wordCount = {} #format wordCount[w(i)] = prob(w(i)), ex wordCount['the'] = 0.454
    count = 0
    wordCount['<s>'] = 0
    wordCount['</s>'] = 0
    #count word
    for sentence in trainingSentences:
        tokens = nltk.word_tokenize(sentence) #break sentence to word
        count += 2 #for <s> and </s>
        wordCount['<s>'] += 1
        wordCount['</s>'] += 1
        for token in tokens: #for the actual word
            tokenLower = token.lower() #to generalize the letter
            if tokenLower in wordCount:
                wordCount[tokenLower] += 1
            else:
                wordCount[tokenLower] = 1
            count += 1
    unigramModel = wordCount
    #count prob
    for word in unigramModel:
        unigramModel[word] = unigramModel[word] / count
    return unigramModel
unigramModel = buildUnigramModel(dataTrain)
print('done')

done


### Bangun Model Bigram

In [32]:
def buildBigramModel(trainingSentences):
    #init
    wordPairCount = {} #format wordPairCount[w(i - 1)][w(i)] = prob(w(i) | w(i - 1)), ex. wordPairCount['nissan']['gt-r'] = 0.38
    wordCount = {} #to count the divider for each word
    wordCount['<s>'] = 0
    wordCount['</s>'] = 0
    wordPairCount['<s>'] = {} #there's no wordPairCount['</s>'] because there's nothing after </s>
    #count word
    for sentence in trainingSentences:
        tokens = nltk.word_tokenize(sentence) #break sentence to word
        counter = 0 #to track the position of the word, to deal with <s> and </s>
        tokenBefore = '<s>' #before the first word
        wordCount['<s>'] += 1
        wordCount['</s>'] += 1
        for token in tokens:
            tokenLower = token.lower() #to generalize the letter
            counter += 1
            if tokenLower not in wordPairCount:
                wordPairCount[tokenLower] = {}
            if counter == len(tokens): #if word at the end of sentence
                if tokenLower in wordPairCount[tokenBefore]: #deal with the word and the word before
                    wordPairCount[tokenBefore][tokenLower] += 1
                else:
                    wordPairCount[tokenBefore][tokenLower] = 1
                if '</s>' in wordPairCount[tokenLower]: #deal with the word and </s> after the word
                    wordPairCount[tokenLower]['</s>'] += 1
                else:
                    wordPairCount[tokenLower]['</s>'] = 1
            else:
                if tokenLower in wordPairCount[tokenBefore]: #deal with the word and the word before
                    wordPairCount[tokenBefore][tokenLower] += 1
                else:
                    wordPairCount[tokenBefore][tokenLower] = 1
            tokenBefore = tokenLower #save the word before
            if tokenLower in wordCount: #count each word
                wordCount[tokenLower] += 1
            else:
                wordCount[tokenLower] = 1
    bigramModel = {}  #init too
    #filling the rest of the model count with 0, so that the "index error" error won't happen
    for wordBefore in wordPairCount.keys():
        for theWord in wordCount.keys():
            if theWord != '<s>': #because <s> is the first, no word before
                if theWord not in wordPairCount[wordBefore]:
                    wordPairCount[wordBefore][theWord] = 0
    #count prob
    for wordBefore in wordPairCount.keys():
        bigramModel[wordBefore] = {}
        for theWord in wordCount.keys():
            if theWord != '<s>': #because <s> is the first, no word before
                bigramModel[wordBefore][theWord] = wordPairCount[wordBefore][theWord] / wordCount[theWord]
    return bigramModel
bigramModel = buildBigramModel(dataTrain)
print('done')

done


### Bangun Fungsi Kata Berikutnya Dengan Probability Tertinggi

In [37]:
#Diberikan sebuah kata, cari kata berikutnya yang memiliki probability tertinggi berdasarkan model bigram
#proportional random from top n word
#param model, a word, n (top n word)
def nextBestWord(bigramModel, currentWord, topn):
    #init
    top = []
    holder = {}
    currentWordLow = currentWord.lower() #to make the case same with the model
    if currentWordLow not in bigramModel: #for OOV word (that is not in the training/model)
        currentWordLow = "_____"
    for i in range(topn):
        maxs = max(bigramModel[currentWordLow].items(), key=operator.itemgetter(1))[0] #get max prob
        holder[maxs] = bigramModel[currentWordLow][maxs] #save the max prob
        top.append(maxs) #save the max word
        bigramModel[currentWordLow].pop(maxs) #pop the max word, so that the next loop get the second max
    for i in range(topn): #put the popped back to the model
        bigramModel[currentWordLow].update( {top[i] : holder[top[i]]} )
    #proportional random
    maks = sum(holder.values())
    pick = random.uniform(0, maks)
    current = 0
    for key, value in holder.items():
        current += value
        if current > pick:
            return key
    #return nextWord
#Lakukan percobaan dengan memberikan sebuah kata, lalu print kata berikutnya
print(nextBestWord(bigramModel,"porsche",3))

setting


### Hitung probabilitas bigram menggunakan metode interpolasi (interpolation)

In [38]:
#count the p(w(i) | w(i - 1)) using the simple interpolation
#params unigram model, bigram model, lambda unigram, lambda bigram, w(i - 1), w(i)
def countInterpolationBigram(unigramModel, bigramModel, lambdauni, lambdabi, wordBefore, theWord):
    if bigramModel[wordBefore][theWord] == 0: #because math.log2(0) crash the program
        return (lambdabi * 0) + (lambdauni * math.log2(unigramModel[theWord]))
    else:
        return (lambdabi * math.log2(bigramModel[wordBefore][theWord])) + (lambdauni * math.log2(unigramModel[theWord]))

### Hitung Perplexity Unigram dan Bigram terhadap Data Testing

In [41]:
def countPerplexity(unigramModel, bigramModel, testSentences):
    perplexity = {} #init
    #unigram
    count1 = 0 #the M
    total1 = 0 #the l
    for sentence in testSentences:
        pos = 0 #track word position
        tokens = nltk.word_tokenize(sentence) #break sentence to words
        for word in tokens:
            pos += 1
            count1 += 1 #for the word
            wordLow = word.lower() #to make the case same with the model
            if word not in unigramModel: #for OOV word (that is not in the training/model)
                wordLow = "_____"
            if pos > 1:
                total1 += math.log2(unigramModel[wordLow])
            elif pos == len(tokens):
                total1 += math.log2(unigramModel[wordLow])
                total1 += math.log2(unigramModel['</s>']) #end of sentence
                count1 += 1 #because </s>
            elif pos == 1:
                count1 += 1 #because <s>
                total1 += math.log2(unigramModel['<s>']) #start of sentence
                total1 += math.log2(unigramModel[wordLow])
    total1 = total1 / count1 #total * 1 / M
    total1 = total1 * -1 #negative total
    perplexity1 = 2 ** total1 #2^-l
    perplexity.update({'unigram' : perplexity1})
    #bigram with simple interpolation (one of the smoothing method)
    lambdauni = 0.2 #lambda unigram model
    lambdabi = 0.8 #lambda bigram model
    count2 = 0 #the M
    total2 = 0 #the l
    for sentence in testSentences:
        pos = 0 #track position
        tokens = nltk.word_tokenize(sentence) #break sentence to words
        wordBefore = '<s>'
        for word in tokens:
            count2 += 1
            pos += 1
            wordLow = word.lower() #to make the case same with the model
            if wordLow not in bigramModel or wordLow not in unigramModel:
                wordLow = "_____" #for OOV word (that is not in the training/model)
            if pos > 0:
                total2 += countInterpolationBigram(unigramModel, bigramModel, lambdauni, lambdabi, wordBefore, wordLow)
            elif pos == len(tokens):
                total2 += countInterpolationBigram(unigramModel, bigramModel, lambdauni, lambdabi, wordBefore, wordLow)
                total2 += countInterpolationBigram(unigramModel, bigramModel, lambdauni, lambdabi, wordLow, '</s>') #end of sentence
                count2 += 1 #for end of sentence (</s>)
            wordBefore = wordLow #save the w(i - 1)
    total2 = total2 / count2 #total * 1 / M
    total2 = total2 * -1 #negative total
    perplexity2 = 2 ** total2 #2^-l
    perplexity.update({'bigram' : perplexity2})
    return perplexity
print(countPerplexity(unigramModel, bigramModel, dataTest))

{'unigram': 702.8664484694253, 'bigram': 12.562666753706731}


### Bangun Fungsi Generate Kalimat

In [45]:
#params, the model, number of word, and top-n for the generator
def generateSentence(model, length, top):
    sentence = '' #init
    wordBefore = '<s>' #very first word
    for i in range(length):
        while(True): #to avoid generating _____ (the OOV dummy model)
            nextWord = nextBestWord(model,wordBefore,top)
            if nextWord != '_____':
                break
        if nextWord == '<s>':
            sentence += ' '
        elif nextWord == "</s>":
            sentence += '.'
        else:
            #for better spacing
            if nextWord != "'" and nextWord != '’' and nextWord != '‘' and wordBefore != '’' and wordBefore != '”' and wordBefore != '(' and nextWord != '”' and nextWord != '"' and nextWord != ',' and nextWord != '(' and nextWord != ')':
                sentence += ' '
            sentence += nextWord
        wordBefore = nextWord
    return sentence
generateSentence(bigramModel, 100, 2)

' thankfully, knocking 150kg off hitting 500kmh at 277mph over 1000bhp/tonne. has wonderful balance but butter-smooth roads will evaporate in 275/35 tyres moulded components such demands for conversion. has whipped up 258bhp at 6000rpm and 630lb ft torque split. has accidentally made possible without losing the puzzle is 2000ps(w10) boasting faster shifts, clutch and 187,000(ss) magneto rheological suspension geometry with circular daytime running light remap to €68 this scenario. has acquired the wake of dieselgate and admire how awesome it 150bhp less controversial bonnet lives a standalone entity formed'